In [1]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm.notebook import tqdm


In [2]:
list_of_urls = []
for i in tqdm(range(1, 100)):
    list_of_urls.append(f'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum={i}')
list_of_urls

  0%|          | 0/99 [00:00<?, ?it/s]

['https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=1',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=2',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=3',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=4',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=5',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=6',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=7',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pgnum=8',
 'https://www.codeproject.com/search.aspx?q=machine+learning&doctypeid=1%3b2%3b3%3b13%3b4%3b5&pgsz=50&pg

In [3]:
headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
        }

In [4]:
list_link = []
for i in tqdm(list_of_urls, desc='Crawling ...'):
    response = requests.get(i, headers=headers)
    sub_soup = BeautifulSoup(response.content, 'html.parser')
    taget = sub_soup.find_all('div', class_='hover-container content-list-item clearfix')
    [list_link.append('https:'+i.find('a')['href']) for i in taget]


Crawling ...:   0%|          | 0/205 [00:00<?, ?it/s]

In [11]:
import requests
from bs4 import BeautifulSoup
import json
from concurrent.futures import ThreadPoolExecutor

session = requests.Session()
session.max_redirects = 50

sum_all = 0
tag = 0
data = []
failed_links = []

def crawl_link(i):
    try:
        redirect_count = 0
        while redirect_count < 50:
            st = session.get(i, headers=headers, timeout=10, allow_redirects=False)
            if st.status_code == 200:
                break
            elif st.status_code == 302:
                redirect_count += 1
                i = st.headers['Location']
            else:
                print(f"Failed to access {i} with status code {st.status_code}")
                break

        if st.status_code == 200:
            result = BeautifulSoup(st.content, 'html.parser')
            content = result.find('div', attrs={'id': 'post-content-wrapper'})
            if content is None:
                return None
            title = result.find('h1', attrs={'data-query': 'post-title'})
            if title is None:
                return None
            datas = {
                "url": i,
                "title": title.text.strip(),
                "content": content.text.strip(),
                "code": f"codeproject_CV_{tag}",
                "language": 'English',
                "tags": "CV"
            }
            return datas
    except requests.exceptions.Timeout:
        print(f"Timeout occurred while accessing {i}")
        return None

# Sử dụng ThreadPoolExecutor để chạy các yêu cầu mạng song song
with ThreadPoolExecutor() as executor:
    # Sử dụng map() để ánh xạ các yêu cầu mạng vào các luồng
    results = list(executor.map(crawl_link, list_link))

for result in results:
    if result is not None:
        data.append(result)
        tag += 1
        sum_all += 1
    else:
        failed_links.append(i)

with open('data6.json', 'r', encoding='utf-8-sig') as f:
    existing_data = json.load(f)

existing_data.extend(data)

with open('data6.json', 'w', encoding='utf-8') as f:
    json.dump(existing_data, f, indent=4, ensure_ascii=False)

print(f"Total completed requests: {sum_all}")


c:\Users\MSI20\AppData\Local\Programs\Python\Python38\lib\threading.py:228: RuntimeWarning: coroutine 'main' was never awaited
  self.release = lock.release
